In [6]:
# | default_exp backend.tools.fs_write_backup

In [9]:
# | export
from agentic.tools.base import Tool
from agentic.schemas import  WriteCommand
from pathlib import Path
import difflib
import shutil
from datetime import datetime
from typing import Dict, Any


In [8]:
# | export
class FsWriteTool(Tool):
    def get_tool_schema(self) -> Dict[str, Any]:
        """Return the OpenAI-compatible schema for fs_write."""
        return {
                "type": "function",
                "function": {
                    "name": "fs_write",
                    "description": "file operations: create, edit, modify files with diff preview, insert at specific lines, and batch operations. Shows changes in git diff format.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "command": {
                                "type": "string", 
                                "enum": ["create", "str_replace", "append", "insert", "prepend", "delete_lines"],
                                "description": "Operation: create (new file), str_replace (replace text), append (add to end), insert (add at line), prepend (add to start), delete_lines (remove lines)"
                            },
                            "path": {"type": "string", "description": "File path"},
                            "file_text": {"type": "string", "description": "Content for create command"},
                            "old_str": {"type": "string", "description": "Text to replace (exact match required)"},
                            "new_str": {"type": "string", "description": "Replacement text"},
                            "insert_line": {"type": "integer", "description": "Line number to insert at (1-based)"},
                            "start_line": {"type": "integer", "description": "Start line for delete_lines"},
                            "end_line": {"type": "integer", "description": "End line for delete_lines"},
                            "create_backup": {"type": "boolean", "description": "Create backup before editing"},
                            "show_diff": {"type": "boolean", "description": "Show diff of changes"},
                            "summary": {"type": "string", "description": "Brief description of the change"}
                        },
                        "required": ["command", "path"]
                    }
                }
            }

    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute fs_write with the provided arguments."""
        try:
            params = FsWriteParams(**kwargs)
            path_obj = Path(params.path)
            original_content = ""
            
            if path_obj.exists():
                try:
                    with open(params.path, 'r', encoding='utf-8', errors='ignore') as f:
                        original_content = f.read()
                except:
                    original_content = "[Binary or unreadable file]"
            
            if params.create_backup and path_obj.exists():
                backup_path = f"{params.path}.backup.{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                shutil.copy2(params.path, backup_path)
            
            new_content = original_content
            
            if params.command == WriteCommand.CREATE:
                file_text = params.file_text or ""
                path_obj.parent.mkdir(parents=True, exist_ok=True)
                new_content = file_text
                
                with open(params.path, 'w', encoding='utf-8') as f:
                    f.write(file_text)
                
                result = {
                    "success": True,
                    "message": f"Created file: {params.path}",
                    "lines_added": len(file_text.splitlines()),
                    "size_bytes": len(file_text.encode('utf-8'))
                }
                
                if params.show_diff and file_text:
                    result["diff"] = self._generate_diff("", file_text, params.path, "created")
                
                return result
            
            elif params.command == WriteCommand.APPEND:
                file_text = params.file_text or ""
                with open(params.path, 'a', encoding='utf-8') as f:
                    f.write(file_text)
                new_content = original_content + file_text
                
                result = {
                    "success": True,
                    "message": f"Appended to file: {params.path}",
                    "lines_added": len(file_text.splitlines())
                }
                
                if params.show_diff:
                    result["diff"] = self._generate_diff(original_content, new_content, params.path, "appended")
                
                return result
            
            elif params.command == WriteCommand.PREPEND:
                file_text = params.file_text or ""
                new_content = file_text + original_content
                with open(params.path, 'w', encoding='utf-8') as f:
                    f.write(new_content)
                
                result = {
                    "success": True,
                    "message": f"Prepended to file: {params.path}",
                    "lines_added": len(file_text.splitlines())
                }
                
                if params.show_diff:
                    result["diff"] = self._generate_diff(original_content, new_content, params.path, "prepended")
                
                return result
            
            elif params.command == WriteCommand.STR_REPLACE:
                old_str = params.old_str or ""
                new_str = params.new_str or ""
                if not old_str:
                    return {"error": "old_str is required for str_replace"}
                
                new_content = original_content.replace(old_str, new_str)
                with open(params.path, 'w', encoding='utf-8') as f:
                    f.write(new_content)
                
                result = {
                    "success": True,
                    "message": f"Replaced text in file: {params.path}",
                    "replacements": original_content.count(old_str)
                }
                
                if params.show_diff:
                    result["diff"] = self._generate_diff(original_content, new_content, params.path, "replaced")
                
                return result
            
            elif params.command == WriteCommand.INSERT:
                if params.insert_line is None or params.file_text is None:
                    return {"error": "insert_line and file_text are required for insert"}
                
                lines = original_content.splitlines(keepends=True)
                insert_idx = max(0, min(params.insert_line - 1, len(lines)))
                lines.insert(insert_idx, params.file_text)
                new_content = ''.join(lines)
                
                with open(params.path, 'w', encoding='utf-8') as f:
                    f.write(new_content)
                
                result = {
                    "success": True,
                    "message": f"Inserted at line {params.insert_line} in file: {params.path}",
                    "lines_added": len(params.file_text.splitlines())
                }
                
                if params.show_diff:
                    result["diff"] = self._generate_diff(original_content, new_content, params.path, "inserted")
                
                return result
            
            elif params.command == WriteCommand.DELETE_LINES:
                if params.start_line is None or params.end_line is None:
                    return {"error": "start_line and end_line are required for delete_lines"}
                
                lines = original_content.splitlines(keepends=True)
                start_idx = max(0, params.start_line - 1)
                end_idx = min(len(lines), params.end_line)
                new_content = ''.join(lines[:start_idx] + lines[end_idx:])
                
                with open(params.path, 'w', encoding='utf-8') as f:
                    f.write(new_content)
                
                result = {
                    "success": True,
                    "message": f"Deleted lines {params.start_line}-{params.end_line} in file: {params.path}",
                    "lines_deleted": end_idx - start_idx
                }
                
                if params.show_diff:
                    result["diff"] = self._generate_diff(original_content, new_content, params.path, "deleted")
                
                return result
            
            return {"error": f"Unknown command: {params.command}"}
            
        except Exception as e:
            return {"error": f"File operation failed: {str(e)}"}

    def _generate_diff(self, original: str, new: str, filepath: str, operation: str) -> str:
        """Generate unified diff for file changes."""
        if original == new:
            return "No changes"
        
        original_lines = original.splitlines(keepends=True)
        new_lines = new.splitlines(keepends=True)
        
        diff_lines = list(difflib.unified_diff(
            original_lines,
            new_lines,
            fromfile=f"a/{filepath}",
            tofile=f"b/{filepath}",
            lineterm=""
        ))
        
        if not diff_lines:
            return "No changes"
        
        diff_output = []
        diff_output.append(f"--- a/{filepath}")
        diff_output.append(f"+++ b/{filepath}")
        
        for line in diff_lines[2:]:
            diff_output.append(line.rstrip())
        
        added_lines = sum(1 for line in diff_lines if line.startswith('+') and not line.startswith('+++'))
        removed_lines = sum(1 for line in diff_lines if line.startswith('-') and not line.startswith('---'))
        
        summary = f"\n📝 Changes: +{added_lines} -{removed_lines} lines ({operation})"
        
        return '\n'.join(diff_output) + summary